# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port mathurin,-19.6833,63.4167,25.05,78,100,6.40,MU,1733865883
1,1,tiksi,71.6872,128.8694,-30.98,99,63,1.45,RU,1733865884
2,2,nueva guinea,11.6876,-84.4562,26.10,85,43,5.44,NI,1733865885
3,3,blackmans bay,-43.0167,147.3167,14.07,72,91,1.34,AU,1733865886
4,4,utrik,11.2278,169.8474,27.57,80,100,9.52,MH,1733865887


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [46]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.25,
    color = "City"
)


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df=city_data_df[(city_data_df["Max Temp"]>15)&
                              (city_data_df["Max Temp"]<30)&
                              (city_data_df["Cloudiness"]==0)&
                              (city_data_df["Humidity"]<60)].reset_index()

# Drop any rows with null values
ideal_weather_df=ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,86,86,kapoeta,4.7717,33.5903,28.77,34,0,5.49,SS,1733865986
1,95,95,lompoc,34.6391,-120.4579,21.62,41,0,2.57,US,1733865997
2,158,158,broken hill,-31.9500,141.4333,17.31,49,0,7.89,AU,1733866077
3,164,164,gwadar,25.1216,62.3254,20.59,41,0,3.61,PK,1733866084
4,176,176,cabo san lucas,22.8909,-109.9124,25.42,52,0,3.09,MX,1733865885
5,207,207,heroica caborca,30.7167,-112.1500,23.12,5,0,6.76,MX,1733866136
6,218,218,castillos,-34.1667,-53.8333,20.11,59,0,3.64,UY,1733865902
7,228,228,valparaiso,-33.0393,-71.6273,22.13,48,0,3.09,CL,1733865602
8,268,268,as sulayyil,20.4607,45.5779,16.40,36,0,3.19,SA,1733866208
9,270,270,abqaiq,25.9371,49.6776,15.24,57,0,4.68,SA,1733865929


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_weather_df.loc[:,["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,kapoeta,SS,4.7717,33.5903,34,
1,lompoc,US,34.6391,-120.4579,41,
2,broken hill,AU,-31.9500,141.4333,49,
3,gwadar,PK,25.1216,62.3254,41,
4,cabo san lucas,MX,22.8909,-109.9124,52,
5,heroica caborca,MX,30.7167,-112.1500,5,
6,castillos,UY,-34.1667,-53.8333,59,
7,valparaiso,CL,-33.0393,-71.6273,48,
8,as sulayyil,SA,20.4607,45.5779,36,
9,abqaiq,SA,25.9371,49.6776,57,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat=hotel_df.loc[index,"Lat"]
    lon=hotel_df.loc[index,"Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapoeta - nearest hotel: Kuleu
lompoc - nearest hotel: Red Roof Inn Lompoc
broken hill - nearest hotel: Ibis Styles
gwadar - nearest hotel: Sadaf Resorts
cabo san lucas - nearest hotel: Comfort Rooms
heroica caborca - nearest hotel: Hotel Posada del Desierto
castillos - nearest hotel: A mi gente
valparaiso - nearest hotel: Ibis Hotel
as sulayyil - nearest hotel: No hotel found
abqaiq - nearest hotel: No hotel found
concepcion - nearest hotel: Hotel One (ex-Biobio)
gualeguaychu - nearest hotel: Yaro
talcahuano - nearest hotel: Club Recreativo Armada de Chile
fortuna - nearest hotel: Comfort Inn & Suites Redwood Country
port augusta - nearest hotel: Hotel Commonwealth


,City,Country,Lat,Lng,Humidity,Hotel Name
0,kapoeta,SS,4.7717,33.5903,34,Kuleu
1,lompoc,US,34.6391,-120.4579,41,Red Roof Inn Lompoc
2,broken hill,AU,-31.9500,141.4333,49,Ibis Styles
3,gwadar,PK,25.1216,62.3254,41,Sadaf Resorts
4,cabo san lucas,MX,22.8909,-109.9124,52,Comfort Rooms
5,heroica caborca,MX,30.7167,-112.1500,5,Hotel Posada del Desierto
6,castillos,UY,-34.1667,-53.8333,59,A mi gente
7,valparaiso,CL,-33.0393,-71.6273,48,Ibis Hotel
8,as sulayyil,SA,20.4607,45.5779,36,No hotel found
9,abqaiq,SA,25.9371,49.6776,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.7,
    color = "City",
    hover_cols=["Hotel Name","Country"],
    size="Humidity"
)


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)